In [17]:
from nemo.lightning import AutoResume

resume = AutoResume(
    resume_if_exists=True,
    resume_ignore_no_cㄇheckpoint=False,
    resume_from_directory="/datasets/soc-20250703225140/nemo_checkpoints/nemotron_49b_super_custom_finetune/2025-07-14_17-42-12/checkpoints/model_name=0--val_loss=0.03-step=449-consumed_samples=900.0-last"
)

SyntaxError: incomplete input (3747645088.py, line 8)

In [ ]:
# %%writefile nemo_inference.py

import torch.distributed
from megatron.core.inference.common_inference_params import CommonInferenceParams
import nemo.lightning as nl
import re

strategy = nl.MegatronStrategy(
    tensor_model_parallel_size=2,
    pipeline_model_parallel_size=1,
    context_parallel_size=1,
    sequence_parallel=False,
    setup_optimizers=False,
    store_optimizer_states=False,
)

trainer = nl.Trainer(
    accelerator="gpu",
    devices=2,
    num_nodes=1,
    strategy=strategy,
    plugins=nl.MegatronMixedPrecision(
        precision="bf16-mixed",
        params_dtype=torch.bfloat16,
        pipeline_dtype=torch.bfloat16,
        autocast_enabled=False,
        grad_reduce_in_fp32=False,
    ),
)

source = {
    "mask": "User",
    "system": "",
    "conversations": [
        {
            "from": "User",
            "value": ""
        },
    ]
}
special_tokens = {
                "system_turn_start": "<extra_id_0>",
                "turn_start": "<extra_id_1>",
                "label_start": "<extra_id_2>",
                "end_of_turn": "\n",
                "end_of_name": "\n",
            }

from nemo.collections.nlp.data.language_modeling.megatron.gpt_sft_chat_dataset import _get_header_conversation_type_mask_role
# Apply prompt template to be the same format as training
header, conversation, data_type, mask_role = _get_header_conversation_type_mask_role(source, special_tokens)
prompts = [conversation]

from nemo.collections.llm import api
results = api.generate(
    path="/datasets/soc-20250703225140/nemo_checkpoints/nemotron_49b_super_custom_finetune/2025-07-17_13-54-48/checkpoints/model_name=0--val_loss=0.02-step=2459-consumed_samples=4920.0-last",
    prompts=prompts,
    trainer=trainer,
    inference_params=CommonInferenceParams(
        temperature=1.0,
        top_p=0,  # greedy decoding
        top_k=1,  # greedy decoding
        num_tokens_to_generate=50,
    ),
    text_only=True,
)
if torch.distributed.get_rank() == 0:
    for i, r in enumerate(results):
        print("=" * 50)
        print(prompts[i])
        print("*" * 50)
        if match:
            print(match.group(0))
        else:
            print(r)
        print("=" * 50)
        print("\n\n")

In [19]:
import inspect
from nemo.collections import llm

## function
# inspect.signature(llm.llama33_nemotron_super_49b)
# inspect.signature(llm.LlamaNemotronModel)
# inspect.signature(llm.llama33_nemotron_super_49b.finetune_recipe)

print(dir(llm.llama33_nemotron_super_49b.finetune_recipe().trainer))
print((llm.llama33_nemotron_super_49b.finetune_recipe().trainer.limit_val_batches))
# print(dir(llm.llama33_nemotron_super_49b.finetune_recipe().resume.resume_from_directory))
# print(dir(llm.llama33_nemotron_super_49b.finetune_recipe().resume)))
# print(dir(llm.llama33_nemotron_super_49b.finetune_recipe().log))
# print(dir(llm.llama33_nemotron_super_49b))
# print(dir(llm.LlamaNemotronModel))

['accelerator', 'accumulate_grad_batches', 'barebones', 'benchmark', 'callbacks', 'check_val_every_n_epoch', 'default_root_dir', 'detect_anomaly', 'deterministic', 'devices', 'enable_checkpointing', 'enable_model_summary', 'enable_progress_bar', 'fast_dev_run', 'gradient_clip_algorithm', 'gradient_clip_val', 'inference_mode', 'limit_predict_batches', 'limit_test_batches', 'limit_train_batches', 'limit_val_batches', 'log_every_n_steps', 'logger', 'max_epochs', 'max_steps', 'max_time', 'min_epochs', 'min_steps', 'num_nodes', 'num_sanity_val_steps', 'overfit_batches', 'plugins', 'precision', 'profiler', 'reload_dataloaders_every_n_epochs', 'strategy', 'sync_batchnorm', 'use_distributed_sampler', 'val_check_interval']
None


In [27]:
import torch 

torch.cuda.empty_cache()

In [ ]:
from nemo.collections import llm
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
import nemo_run as run
from nemo import lightning as nl

# checkpoint_path="/datasets/soc-20250703225140/nemo_checkpoints/nemotron_49b_super_custom_finetune/2025-07-14_00-47-12/checkpoints/model_name=0--val_loss=0.03-step=499-consumed_samples=1000.0-last/weights/"

# from nemo.collections import llm
# llm.import_ckpt(model=llm.LlamaNemotronModel(llm.Llama33NemotronSuper49BConfig()), source=checkpoint_path)

# llm.LlamaNemotronModel.load_from_checkpoint(checkpoint_path=checkpoint_path)

from megatron.core.inference.common_inference_params import CommonInferenceParams

sft_ckpt_path=str(next((d for d in Path("/datasets/soc-20250703225140/nemo_checkpoints/nemotron_49b_super_custom_finetune/2025-07-14_00-47-12/checkpoints/").iterdir() if d.is_dir() and d.name.endswith("-last")), None))
print("We will load SFT checkpoint from:", sft_ckpt_path)



def trainer() -> run.Config[nl.Trainer]:
    strategy = run.Config(
        nl.MegatronStrategy,
        tensor_model_parallel_size=2,
    )
    trainer = run.Config(
        nl.Trainer,
        accelerator="gpu",
        devices=2,
        num_nodes=1,
        strategy=strategy,
    )
    return trainer

prompts = [
    "How many r's are in the word 'strawberry'?",
    "Which number is bigger? 10.119 or 10.19?",
]

def configure_inference():
    return run.Partial(
        llm.generate,
        path=str(sft_ckpt_path),
        trainer=trainer(),
        prompts=prompts,
        inference_params=CommonInferenceParams(num_tokens_to_generate=8192, top_p=0.95),
        output_path="sft_prediction.jsonl",
    )


def local_executor_torchrun(nodes: int = 1, devices: int = 2) -> run.LocalExecutor:
    # Env vars for jobs are configured here
    env_vars = {
        "TORCH_NCCL_AVOID_RECORD_STREAMS": "1",
        "NCCL_NVLS_ENABLE": "0",
        "NVTE_DP_AMAX_REDUCE_INTERVAL": "0",
        "NVTE_ASYNC_AMAX_REDUCTION": "1",
    }

    executor = run.LocalExecutor(ntasks_per_node=devices, launcher="torchrun", env_vars=env_vars)

    return executor

if __name__ == '__main__':
    run.run(configure_inference(), executor=local_executor_torchrun())


In [7]:
print("He's".replace("'", "\\'"))

He's


In [21]:
# import data_process.guideline_names_extraction
# get_guideline_name()
import os
cpic_guideline_name_list = os.listdir("./data_process/Guidelines")
cpic_guideline_name_list[0]

'Clinical Pharmacogenetics Implementation Consortium (CPIC) Guidelines for Ivacaftor Therapy in the Context of CFTR Genotype (March 2014).pdf'

In [11]:
## 最重要程式碼
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
convert_nemo_dataset.py

將 dataset.json 轉成 NeMo Chat-SFT 所需的
training.jsonl / validation.jsonl / test.jsonl（三檔）。

特點
1. 徹底清理所有隱藏換行 ( \n, \r, U+2028, U+2029 )
2. 對 dict / list value 先做 json.dumps，保證單行
3. 每筆寫入前以 json.dumps() 驗證；含換行就強制丟棄
4. 寫檔完成後再次深度掃描，若仍有殘留換行即拋錯
"""

import json
import os
import random
import re
import sys
from pathlib import Path
from typing import Any, Dict, List

# --------------------------------------------------------------------
# 1.  基本設定
# --------------------------------------------------------------------
SRC_JSON         = "/datasets/soc-20250703225140/dataset/dataset.json"
OUT_DIR          = "/datasets/soc-20250703225140/dataset_split"
SPLIT_RATIOS     = (0.70, 0.15, 0.15)  # train / val / test
RANDOM_SEED      = 42
NEWLINES         = ("\n", "\r", "\u2028", "\u2029")       # 所有可能殘留的換行符

# --------------------------------------------------------------------
# 2.  工具函式
# --------------------------------------------------------------------
def sanitize(text: Any) -> str:
    """將任意物件轉為『單行、安全字串』"""
    s = str(text)
    # 換行符 → 空白
    for ch in NEWLINES:
        s = s.replace(ch, " ")
    # 其他空白壓縮
    s = re.sub(r"\s+", " ", s).strip()
    # 跳脫雙引號
    return s.replace('"', r'\"')


def format_answer(ans: Any) -> str:
    """
    把 answer 轉成 <QUERYCALL>[ ... ]</QUERYCALL>
      - 若為 dict：逐 key/value 處理
      - 其他型別：視為拒答文字，直接 sanitize
    """
    if not isinstance(ans, dict):
        return sanitize(ans)

    parts = []
    for k, v in ans.items():
        if isinstance(v, (dict, list)):
            v = json.dumps(v, separators=(",", ":"))  # 巢狀資料 → 壓成單行 JSON
        parts.append(f'{sanitize(k)}="{sanitize(v)}"')
    return f"<QUERYCALL>[{', '.join(parts)}]</QUERYCALL>"


def has_hidden_newline(s: str) -> bool:
    """判斷字串是否仍含 NEWLINES 中的任一字元"""
    return any(ch in s for ch in NEWLINES)


def deep_scan_jsonl(path: Path) -> List[int]:
    """
    掃描 .jsonl，回傳『仍含隱藏換行符』的行號清單
    備註：使用 newline="" 讀檔，才不會被 Python 自動轉譯
    """
    bad_lines = []
    with path.open("r", encoding="utf-8", newline="") as f:
        for ln, raw in enumerate(f, 1):
            # 去除真正的行尾
            if raw.endswith("\r\n"):
                core = raw[:-2]
            elif raw.endswith("\n"):
                core = raw[:-1]
            else:
                core = raw
            if has_hidden_newline(core):
                bad_lines.append(ln)
    return bad_lines


# --------------------------------------------------------------------
# 3.  主要流程
# --------------------------------------------------------------------
def convert_dataset(
    src_json: str,
    out_dir: str,
    split: tuple = SPLIT_RATIOS,
    seed: int = RANDOM_SEED,
) -> None:
    random.seed(seed)

    # 讀入原始 JSON
    with open(src_json, encoding="utf-8") as f:
        data = json.load(f)
    if not isinstance(data, list):
        raise ValueError("輸入檔案必須是 JSON list。")

    processed: List[Dict] = []
    skipped = 0

    for idx, item in enumerate(data, 1):
        if not (isinstance(item, dict) and "question" in item and "answer" in item):
            skipped += 1
            continue

        record = {
            "system": "",
            "mask": "User",
            "conversations": [
                {"from": "User",      "value": sanitize(item["question"])},
                {"from": "Assistant", "value": format_answer(item["answer"])},
            ],
        }

        # 寫入前再次驗證
        line = json.dumps(record, ensure_ascii=False)
        if has_hidden_newline(line):
            print(f"⚠️  跳過第 {idx} 筆：仍含隱藏換行符")
            skipped += 1
            continue

        processed.append(record)

    print(f"✔ 轉換完成：{len(processed)} 筆（跳過 {skipped} 筆）")

    # 隨機打亂
    random.shuffle(processed)

    # 切分
    n        = len(processed)
    n_train  = int(n * split[0])
    n_val    = int(n * split[1])

    splits = {
        "training.jsonl":   processed[:n_train],
        "validation.jsonl": processed[n_train:n_train + n_val],
        "test.jsonl":       processed[n_train + n_val:],
    }

    # 輸出
    Path(out_dir).mkdir(parents=True, exist_ok=True)
    for fname, recs in splits.items():
        path = Path(out_dir) / fname
        with path.open("w", encoding="utf-8", newline="\n") as f:
            for r in recs:
                f.write(json.dumps(r, ensure_ascii=False) + "\n")

        # 寫完後做深度掃描
        bad = deep_scan_jsonl(path)
        if bad:
            raise RuntimeError(f"{fname} 存在隱藏換行符，壞行號：{bad[:10]}")
        print(f"  → {fname:<17} {len(recs)} 行  （驗證通過）")

    print("\n🎉  全部完成！可直接交給 NeMo 進行 fine-tune。")



convert_dataset(SRC_JSON, OUT_DIR)


✔ 轉換完成：4391 筆（跳過 0 筆）
  → training.jsonl    3073 行  （驗證通過）
  → validation.jsonl  658 行  （驗證通過）
  → test.jsonl        660 行  （驗證通過）

🎉  全部完成！可直接交給 NeMo 進行 fine-tune。


In [1]:
!pip install --upgrade lightning-utilities

DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/lightning_utilities-0.14.0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/deep_ep-1.0.0+a84a248-py3.12-linux-x86_64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/looseversion-1.3.0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/nvfuser-0.2.26a0+c5e1555-py3.12-linux-x86

In [9]:
import os
os.environ['HF_HOME']='/datasets/soc-20250703225140/'
os.environ['HF_HUB_CACHE']='/datasets/soc-20250703225140/'
os.environ['TRANSFORMERS_CACHE']='/datasets/soc-20250703225140/'
os.environ['NEMO_HOME']='/datasets/soc-20250703225140/'

In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("nvidia/Llama-3_3-Nemotron-Super-49B-v1")

In [ ]:
dir(tokenizer)

In [12]:
from nemo.collections import llm
llm.import_ckpt(model=llm.LlamaNemotronModel(llm.Llama33NemotronSuper49BConfig()), source='hf://nvidia/Llama-3_3-Nemotron-Super-49B-v1')

[NeMo W 2025-07-10 23:17:15 nemo_logging:405] /usr/local/lib/python3.12/dist-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
      warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
    


 $NEMO_MODELS_CACHE=/datasets/soc-20250703225140/models 

Imported Checkpoint
├── context/
│   ├── artifacts/
│   │   └── generation_config.json
│   ├── nemo_tokenizer/
│   │   ├── special_tokens_map.json
│   │   ├── tokenizer.json
│   │   └── tokenizer_config.json
│   ├── io.json
│   └── model.yaml
└── weights/
    ├── .metadata
    ├── __0_0.distcp
    ├── __0_1.distcp
    ├── common.pt
    └── metadata.json

PosixPath('/datasets/soc-20250703225140/models/nvidia/Llama-3_3-Nemotron-Super-49B-v1')

In [ ]:
dir(llm.Llama33NemotronSuper49BConfig)

In [25]:
import inspect
from nemo.collections import llm

inspect.signature(llm.llama33_nemotron_super_49b.finetune_recipe)

<Signature (dir: Optional[str] = None, name: str = 'default', num_nodes: int = 1, num_gpus_per_node: int = 8, peft_scheme: Optional[str] = 'lora', seq_length: Optional[int] = None, packed_sequence: Optional[bool] = None, performance_mode: bool = False) -> nemo_run.config.Partial>

In [26]:
from nemo.collections import llm
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from torch.utils.data import Dataset, DataLoader

checkpoint_path="/datasets/soc-20250703225140/models--nvidia--Llama-3_3-Nemotron-Super-49B-v1"


recipe = llm.llama33_nemotron_super_49b.finetune_recipe(
    name="llama33_nemotron_super_49b_finetuning",
    dir=checkpoint_path,
    num_nodes=2,
    num_gpus_per_node=1,
    
    peft_scheme='lora',  # 'lora', 'none'
    packed_sequence=False,
)



In [ ]:
dir(recipe.data)

In [4]:
import json
import os
import ast # 導入 ast 模組

def read_qa_pairs_from_file() -> list:
    """
    從指定路徑的檔案中讀取問題-答案配對。
    假設檔案內容是一個包含多個字典的 JSON 陣列，每個字典有 "question" 和 "answer" 鍵。

    Args:
        file_path (str): 資料檔案的路徑。

    Returns:
        list: 包含問題-答案字典的列表。

    Raises:
        FileNotFoundError: 如果檔案不存在。
        ValueError: 如果檔案內容不是預期的列表或字典格式。
    """
    # 移除這裡的虛擬資料檔創建邏輯，讓函式專注於讀取
    # 這兩個路徑應該作為參數傳入 read_qa_pairs_from_file 或在外部定義
    negative_sample_path = "/datasets/soc-20250703225140/dataset/negative_sample.txt"
    creative_sample_path = "/datasets/soc-20250703225140/dataset/creative_dataset.txt"
    
    negative_list = []
    creative_dataset = None
    
    # 從 negative_sample.txt 讀取，使用 ast.literal_eval
    with open(negative_sample_path, 'r', encoding='utf-8') as f:
        negative_list = ast.literal_eval(f.read())
    
    # 從 creative_dataset.txt 讀取，使用 json.load
    with open(creative_sample_path, "r", encoding='utf-8') as f:
        creative_dataset = json.load(f)

    creative_list = []
    for creative_data_item in creative_dataset: # 修改變數名以避免混淆
        # 假設 creative_data_item 是 {'key': ['qa1', 'qa2']} 的形式
        for key, value_list in creative_data_item.items():
            creative_list.extend(value_list) # 使用 extend 添加列表中的所有元素
        
    qa_pairs = negative_list + creative_list

    if not isinstance(qa_pairs, list) or not all(isinstance(item, dict) and "question" in item and "answer" in item for item in qa_pairs):
        raise ValueError("檔案內容不是有效的 Q&A 字典列表格式。請確認每個項目都是包含 'question' 和 'answer' 鍵的字典。")

    return qa_pairs



In [3]:
# negative_list[0]

{'question': 'What is the CPIC-recommended dose for metformin in a patient with the CYP2D6 *4/*4 genotype?',
 'answer': 'No CPIC guideline information available.'}

In [14]:
# creative_data[0]
# creative_list = []
# for creative_data in creative_dataset:
#     for _, value in creative_data.items():
#         creative_list += value

In [15]:
# creative_list[0]

{'question': 'What is the recommended ivacaftor dose for a patient with the CFTR G551D mutation?',
 'answer': {'Drug Name': 'ivacaftor',
  'Gene Name': 'CFTR',
  'CPIC Guideline Name': 'Clinical Pharmacogenetics Implementation Consortium (CPIC) Guidelines for Ivacaftor Therapy in the Context of CFTR Genotype (March 2014).pdf',
  'Content to Search': 'Recommended ivacaftor dosage for patients with the CFTR G551D genotype.'}}

In [16]:
# qa_dataset = creative_list + negative_list

In [5]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer # 假設你會用 Hugging Face 的分詞器

class QADataset(Dataset):
    def __init__(self, qa_pairs: list, tokenizer: AutoTokenizer, max_length: int = 512):
        """
        初始化自定義的問答資料集。

        Args:
            qa_pairs (list): 從檔案讀取的問題-答案字典列表。
            tokenizer (AutoTokenizer): 用於處理文本的 Hugging Face 分詞器。
            max_length (int): 序列的最大長度，用於分詞器的截斷。
        """
        self.qa_pairs = qa_pairs
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        # 返回資料集中的問題-答案對的總數
        return len(self.qa_pairs)

    def __getitem__(self, idx):
        # 根據索引獲取一個單個樣本
        item = self.qa_pairs[idx]
        question = item["question"]
        answer = item["answer"]

        # 這裡返回原始字串，分詞和填充的步驟將在 `collate_fn` 中處理。
        return {"question": question, "answer": answer}

In [6]:
from torch.utils.data import Dataset, DataLoader
def custom_collate_fn(batch: list, tokenizer: AutoTokenizer, max_length: int):
    """
    自定義的批次化函式。將多個 Q&A 樣本處理成模型可接受的單個批次。

    Args:
        batch (list): 包含多個 `__getitem__` 返回的字典（例如 [{"question": "...", "answer": "..."}, ...]）。
        tokenizer (AutoTokenizer): Hugging Face 分詞器。
        max_length (int): 序列的最大長度。

    Returns:
        dict: 包含 'input_ids', 'attention_mask' 和 'labels' 的字典，適合模型訓練。
    """
    questions = [item["question"] for item in batch]
    answers = [item["answer"] for item in batch]

    # 將問題和答案組合成一個用於因果語言建模的訓練格式。
    # 這種格式常見於微調模型以生成答案。
    # 通常的格式為："問題: [你的問題]\n答案: [你的答案]<eos_token>"
    texts_to_tokenize = [f"問題: {q}\n答案: {a}{tokenizer.eos_token}" for q, a in zip(questions, answers)]

    # 對整個批次的文本進行分詞和填充
    tokenized_batch = tokenizer(
        texts_to_tokenize,
        max_length=max_length, # 將所有序列填充或截斷到這個長度
        truncation=True,       # 如果序列超過 max_length，則截斷
        padding="max_length",  # 填充到 max_length
        return_tensors="pt"    # 返回 PyTorch 張量
    )

    # 對於因果語言模型 (Causal Language Model)，通常 `input_ids` 也用作 `labels`。
    # 模型會在內部將 `labels` 偏移一個位置來進行下一個詞的預測。
    # 我們複製一份，因為 labels 可能會被修改 (例如 -100 填充)。
    labels = tokenized_batch["input_ids"].clone()

    return {
        "input_ids": tokenized_batch["input_ids"],
        "attention_mask": tokenized_batch["attention_mask"],
        "labels": labels,
    }

In [7]:
def a_function_that_configures_your_custom_dataset(gbs: int, mbs: int, seq_length: int, tokenizer: AutoTokenizer) -> DataLoader:
    """
    配置並返回一個用於 NeMo Recipe 的客製化 DataLoader。

    Args:
        gbs (int): Global Batch Size (總批次大小)。
        mbs (int): Micro Batch Size (每個 GPU 的批次大小)。這將作為 DataLoader 的 batch_size。
        seq_length (int): 序列最大長度。
        tokenizer (AutoTokenizer): Hugging Face 分詞器。
        qa_file_path (str): 你的 Q&A 數據檔案的路徑。

    Returns:
        torch.utils.data.DataLoader: 配置好的數據載入器。
    """
    print(f"\nConfiguring DataLoader with: Global Batch Size={gbs}, Micro Batch Size={mbs}, Sequence Length={seq_length}")
    
    # 讀取你的 Q&A 數據
    qa_pairs = read_qa_pairs_from_file()
    
    # 實例化你的客製化數據集
    custom_dataset_instance = QADataset(qa_pairs=qa_pairs, tokenizer=tokenizer, max_length=seq_length)
    print(f"Custom dataset created with {len(custom_dataset_instance)} samples.")
    
    # 創建 DataLoader 實例
    # DataLoader 的 batch_size 參數直接對應每個 GPU 的 micro batch size (mbs)
    custom_dataloader = DataLoader(
        custom_dataset_instance,
        batch_size=mbs,
        shuffle=True, # 訓練時通常需要打亂數據
        num_workers=0, # 在 Jupyter/Colab 環境中，0 通常是安全的選擇
        pin_memory=True, # 啟用 pin_memory 可以加速數據傳輸到 GPU
        collate_fn=lambda b: collate_fn(b, tokenizer=tokenizer, max_length=seq_length)
    )
    print(f"DataLoader created with {len(custom_dataloader)} batches (each of size {mbs}).")
    return custom_dataloader



In [8]:
# --- 範例使用方式 (模擬 NeMo Recipe 和 run.run) ---
# if __name__ == "__main__":
# --- 必要設定與檔案定義 ---

# 設定快取目錄 (確保有寫入權限)

cache_dir = '/datasets/soc-20250703225140/'
os.environ['HF_HOME'] = cache_dir
os.environ['HF_HUB_CACHE'] = cache_dir
os.environ['TRANSFORMERS_CACHE'] = cache_dir
os.environ['NEMO_HOME']=cache_dir
"""
export HF_HOME=/datasets/soc-20250703225140/
export HF_HUB_CACHE=/datasets/soc-20250703225140/
export TRANSFORMERS_CACHE=/datasets/soc-20250703225140/
export NEMO_HOME=/datasets/soc-20250703225140/
"""



# --- 載入分詞器 ---
model_name="nvidia/Llama-3_3-Nemotron-Super-49B-v1"
try:
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        print(f"分詞器的 pad_token 已設定為 eos_token: {tokenizer.pad_token_id}")
except Exception as e:
    print(f"錯誤：無法載入分詞器 '{tokenizer_model_id}'。{e}")
    exit(1)

# --- 模擬 NeMo Recipe 參數 ---
# 這些參數通常從 NeMo Recipe 或其模型配置中獲取
# 確保這些值與你的實際訓練配置相符
gbs_param = 8   # Global Batch Size
mbs_param = 4   # Micro Batch Size per GPU


# --- 呼叫 a_function_that_configures_your_custom_dataset 來獲取 DataLoader ---
dataloader = a_function_that_configures_your_custom_dataset(
    gbs=gbs_param,
    mbs=mbs_param,
    seq_length=recipe.model.config.seq_length,
    tokenizer=tokenizer,
)



分詞器的 pad_token 已設定為 eos_token: 128009

Configuring DataLoader with: Global Batch Size=8, Micro Batch Size=4, Sequence Length=2048
Custom dataset created with 4391 samples.
DataLoader created with 1098 batches (each of size 4).


In [9]:
cache_dir = '/datasets/soc-20250703225140/'
# os.makedirs(my_hf_cache_dir, exist_ok=True)
os.environ['HF_HOME'] = cache_dir
os.environ['HF_HUB_CACHE'] = cache_dir
os.environ['TRANSFORMERS_CACHE'] = cache_dir
os.environ['NEMO_HOME']=cache_dir
"""
export HF_HOME=/datasets/soc-20250703225140/
export HF_HUB_CACHE=/datasets/soc-20250703225140/
export TRANSFORMERS_CACHE=/datasets/soc-20250703225140/
export NEMO_HOME=/datasets/soc-20250703225140/
"""



# --- 載入分詞器 ---
model_name="nvidia/Llama-3_3-Nemotron-Super-49B-v1"
try:
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        print(f"分詞器的 pad_token 已設定為 eos_token: {tokenizer.pad_token_id}")
except Exception as e:
    print(f"錯誤：無法載入分詞器 '{tokenizer_model_id}'。{e}")
    exit(1)

# --- 模擬 NeMo Recipe 參數 ---
# 這些參數通常從 NeMo Recipe 或其模型配置中獲取
# 確保這些值與你的實際訓練配置相符
gbs_param = 8   # Global Batch Size
mbs_param = 4   # Micro Batch Size per GPU


# --- 呼叫 a_function_that_configures_your_custom_dataset 來獲取 DataLoader ---
dataloader = a_function_that_configures_your_custom_dataset(
    gbs=gbs_param,
    mbs=mbs_param,
    seq_length=recipe.model.config.seq_length,
    tokenizer=tokenizer,
)



分詞器的 pad_token 已設定為 eos_token: 128009

Configuring DataLoader with: Global Batch Size=8, Micro Batch Size=4, Sequence Length=2048
Custom dataset created with 4391 samples.
DataLoader created with 1098 batches (each of size 4).


In [10]:
dataloader = a_function_that_configures_your_custom_dataset(
    gbs=gbs_param,
    mbs=mbs_param,
    tokenizer=tokenizer,
    seq_length=recipe.model.config.seq_length,
)
recipe.data = dataloader


Configuring DataLoader with: Global Batch Size=8, Micro Batch Size=4, Sequence Length=2048
Custom dataset created with 4391 samples.
DataLoader created with 1098 batches (each of size 4).


In [11]:
import nemo_run as run

recipe.trainer.strategy.tensor_model_parallel_size = 2
recipe.trainer.strategy.pipeline_model_parallel_size = 1

run.run(recipe, executor=run.LocalExecutor())

─── Entering Experiment nemo.collections.llm.api.finetune with id: nemo.collections.llm.api.finetune_1751990357 ───

UnserializableValueError: Unserializable value <torch.utils.data.dataloader.DataLoader object at 0x7f18c47ebce0> of type <class 'torch.utils.data.dataloader.DataLoader'>. Error occurred at path '<root>.data'.")

In [ ]:
import os
import torch
import json
from transformers import AutoTokenizer
from torch.utils.data import Dataset, DataLoader
from nemo.collections import llm
import nemo_run as run
from nemo.collections.common.data.datamodule import DataModule as NeMoDataModule

# --- 0. Setup: Define necessary paths and ensure directories exist ---
my_hf_cache_dir = "/datasets/soc-20250703225140/" # Your specified cache dir
os.makedirs(my_hf_cache_dir, exist_ok=True)
os.environ['HF_HOME'] = my_hf_cache_dir
os.environ['HF_HUB_CACHE'] = my_hf_cache_dir
os.environ['TRANSFORMERS_CACHE'] = my_hf_cache_dir
os.environ['NEMO_HOME'] = my_hf_cache_dir

checkpoint_dir = "/datasets/soc-20250703225140/nemo_checkpoints" # Example, ensure this exists
os.makedirs(checkpoint_dir, exist_ok=True)

qa_data_file = "/datasets/soc-20250703225140/dataset/negative_sample.txt" # Your data file path

# --- 1. Custom Dataset Class ---
class QADataset(Dataset):
    def __init__(self, qa_pairs: list, tokenizer: AutoTokenizer, max_length: int = 512):
        self.qa_pairs = qa_pairs
        self.tokenizer = tokenizer
        self.max_length = max_length
    def __len__(self):
        return len(self.qa_pairs)
    def __getitem__(self, idx):
        item = self.qa_pairs[idx]
        question = item["question"]
        answer = item["answer"]
        return {"question": question, "answer": answer}

# --- 2. Custom Collate Function ---
def collate_fn(batch: list, tokenizer: AutoTokenizer, max_length: int):
    questions = [item["question"] for item in batch]
    answers = [item["answer"] for item in batch]
    texts_to_tokenize = [f"問題: {q}\n答案: {a}{tokenizer.eos_token}" for q, a in zip(questions, answers)]
    tokenized_batch = tokenizer(
        texts_to_tokenize,
        max_length=max_length,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )
    labels = tokenized_batch["input_ids"].clone()
    return {
        "input_ids": tokenized_batch["input_ids"],
        "attention_mask": tokenized_batch["attention_mask"],
        "labels": labels,
    }

# --- 3. Read QA Pairs From File (Adjusted for your specific files) ---
def read_qa_pairs_from_file() -> list: # Removed file_path arg as paths are hardcoded
    negative_sample_path = "/datasets/soc-20250703225140/dataset/negative_sample.txt"
    creative_sample_path = "/datasets/soc-20250703225140/dataset/creative_dataset.txt" # Assuming this is JSON

    negative_list = []
    creative_dataset = None

    # Read negative_sample.txt (assumed to be Python list string)
    if os.path.exists(negative_sample_path):
        with open(negative_sample_path, 'r', encoding='utf-8') as f:
            negative_list = ast.literal_eval(f.read())
    else:
        print(f"Warning: {negative_sample_path} not found. Skipping negative samples.")

    # Read creative_dataset.txt (assumed to be JSON)
    if os.path.exists(creative_sample_path):
        with open(creative_sample_path, "r", encoding='utf-8') as f:
            creative_dataset = json.load(f)
    else:
        print(f"Warning: {creative_sample_path} not found. Skipping creative dataset.")
        creative_dataset = []

    creative_list = []
    if creative_dataset:
        for creative_data_item in creative_dataset:
            for _, value_list in creative_data_item.items():
                creative_list.extend(value_list)
        
    qa_pairs = negative_list + creative_list

    if not isinstance(qa_pairs, list) or not all(isinstance(item, dict) and "question" in item and "answer" in item for item in qa_pairs):
        raise ValueError("Combined data is not a valid list of Q&A dictionaries.")

    return qa_pairs

# --- 4. Custom DataModule (as previously defined) ---
class CustomQAPairDataModule(NeMoDataModule):
    def __init__(self,
                 qa_file_path: str, # Now takes the main combined QA file path
                 tokenizer: AutoTokenizer, # Still pass tokenizer instance
                 max_length: int,
                 gbs: int,
                 mbs: int,
                 num_workers: int = 0,
                 shuffle: bool = True,
                 ):
        super().__init__()
        self.qa_file_path = qa_file_path
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.gbs = gbs
        self.mbs = mbs
        self.num_workers = num_workers
        self.shuffle = shuffle
        
        self.qa_pairs = None # Data will be loaded in setup

    def setup(self, stage: str = None):
        # Data loading now happens within the data module's setup
        # It's better to pass the full list of Q&A pairs to the DataModule init
        # then to load it inside, if the list is already constructed in the main script.
        # But for this example, we keep the file path here.
        self.qa_pairs = read_qa_pairs_from_file() # Calls the global function
        print(f"[DataModule] Loaded {len(self.qa_pairs)} QA pairs in setup stage.")
        
        self.train_dataset = QADataset(
            qa_pairs=self.qa_pairs,
            tokenizer=self.tokenizer,
            max_length=self.max_length
        )

    def train_dataloader(self):
        custom_collater = lambda batch: collate_fn(batch, tokenizer=self.tokenizer, max_length=self.max_length)
        return DataLoader(
            self.train_dataset,
            batch_size=self.mbs,
            shuffle=self.shuffle,
            num_workers=self.num_workers,
            pin_memory=True,
            collate_fn=custom_collater,
        )

# --- 5. Function to configure DataModule CFG ---
def a_function_that_configures_your_custom_dataset_cfg(gbs: int, mbs: int, seq_length: int, tokenizer_model_id: str, qa_file_path: str) -> dict:
    return {
        "_target_": f"{__name__}.CustomQAPairDataModule",
        "qa_file_path": qa_file_path,
        "tokenizer": {"_target_": "transformers.AutoTokenizer.from_pretrained", "pretrained_model_name_or_path": tokenizer_model_id, "trust_remote_code": True, "cache_dir": os.getenv('HF_HOME')}, # Ensure tokenizer loads to correct cache
        "max_length": seq_length,
        "gbs": gbs,
        "mbs": mbs,
        "num_workers": 0,
        "shuffle": True,
    }


# --- Main execution block ---
if __name__ == "__main__":
    import ast # Need to import ast here as well if not already
    
    # --- Necessary setup and file definitions ---
    cache_dir = '/datasets/soc-20250703225140/'
    # os.makedirs(cache_dir, exist_ok=True) # Assuming this is handled by PrimeHub
    os.environ['HF_HOME'] = cache_dir
    os.environ['HF_HUB_CACHE'] = cache_dir
    os.environ['TRANSFORMERS_CACHE'] = cache_dir
    os.environ['NEMO_HOME'] = cache_dir
    print(f"Hugging Face and NeMo cache directory set to: {cache_dir}")

    checkpoint_path = "/datasets/soc-20250703225140/nemo_checkpoints"
    os.makedirs(checkpoint_path, exist_ok=True)
    
    # --- Load Tokenizer ---
    model_name = "nvidia/Llama-3_3-Nemotron-Super-49B-v1"
    try:
        tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, cache_dir=cache_dir)
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
            print(f"分詞器的 pad_token 已設定為 eos_token: {tokenizer.pad_token_id}")
    except Exception as e:
        print(f"錯誤：無法載入分詞器 '{model_name}'。請檢查網路連線或 HFToken。錯誤: {e}")
        exit(1)

    # --- Simulate NeMo Recipe Parameters ---
    # These parameters typically come from the NeMo Recipe or its model configuration
    num_nodes_actual = 1       # You have 1 node
    num_gpus_per_node_actual = 2 # You have 2 A100 80GB GPUs on this node

    global_batch_size_recipe = 8   # Global Batch Size
    micro_batch_size_per_gpu_recipe = 4 # Micro Batch Size per GPU (adjust based on OOM tests)

    # You can specify this directly or derive from a model config if available
    model_seq_length_from_recipe = 2048 # Example: 2048 or 4096, adjust based on model/GPU

    # --- Instantiate NeMo Recipe ---
    recipe = llm.llama33_nemotron_super_49b.finetune_recipe(
        name="llama33_nemotron_super_49b_finetuning",
        dir=checkpoint_path,
        num_nodes=num_nodes_actual,
        num_gpus_per_node=num_gpus_per_node_actual,
        peft_scheme='lora',
        packed_sequence=False,
        global_batch_size=global_batch_size_recipe,
        micro_batch_size_per_gpu=micro_batch_size_per_gpu_recipe,
        seq_length=model_seq_length_from_recipe,
    )
    print("\nNeMo Recipe 實例已創建。")

    # --- CRITICAL FIX: Set Model Parallelism Parameters ---
    # These parameters must be set AFTER the recipe is instantiated
    # but BEFORE it is run.
    # For 2 GPUs, TP=2, PP=1 is the only valid combination for optimal use.
    recipe.trainer.strategy.tensor_model_parallel_size = 2
    recipe.trainer.strategy.pipeline_model_parallel_size = 1 # Assuming no pipeline parallelism
    # If context parallelism is implicitly used, ensure its size * (TP*PP) divides total GPUs.
    # For Llama 3 models, context_parallel_size is often 1.

    print(f"設定模型並行參數: tensor_model_parallel_size={recipe.trainer.strategy.tensor_model_parallel_size}, pipeline_model_parallel_size={recipe.trainer.strategy.pipeline_model_parallel_size}")


    # --- Override the data argument ---
    custom_datamodule_cfg = a_function_that_configures_your_custom_dataset_cfg(
        gbs=global_batch_size_recipe,
        mbs=micro_batch_size_per_gpu_recipe,
        seq_length=model_seq_length_from_recipe,
        tokenizer_model_id=model_name, # Pass tokenizer_model_id, not the tokenizer instance
        qa_file_path=qa_data_file
    )
    recipe.data = custom_datamodule_cfg
    print("NeMo Recipe's data argument has been successfully overridden with your custom DataModule configuration.")

    # --- Run the NeMo Recipe ---
    print("\n嘗試運行 NeMo Recipe...")
    try:
        run.run(recipe, executor=run.LocalExecutor())
        print("\nNeMo Recipe 執行成功！")
    except Exception as e:
        print(f"\n運行 NeMo Recipe 時發生錯誤：{e}")
        print("請檢查您的 NeMo 配置、數據路徑、以及 GPU 資源是否匹配。")